In [ ]:
import numpy as np
import pandas as pd
import os                    
import matplotlib.pyplot as plt
from fbprophet import Prophet
import calendar
import datetime
from datetime import datetime, timedelta
from datetime import date
import math
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

### Import data

Data already treated in previous script is imported here.

In [ ]:
df_d_sales_full = pd.read_csv("./data/full_daily_series.csv")
df_all_sales = pd.read_csv("./data/df_all_monthly_sales.csv")
df_m_sales = pd.read_csv("./data/full_monthly_series.csv")
df_calendar = pd.read_csv("./data/calendar.csv")
df_working_dd = pd.read_csv("./data/working_days.csv")
df_actual_sales = pd.read_csv("./data/actual_sales_series.csv")
df_lo_sales = pd.read_csv("./data/lo_series.csv")

In [ ]:
df_actual_sales = pd.read_csv("./data/actual_sales_series.csv")

# PROPHET

We prepare the data to apply Prophet.

In [ ]:
df_pr = df_d_sales_full[["date", "daily_sales"]]
df_pr.columns = ['ds','y'] # redefine columns names

In [ ]:
# Complete df_pr in order to have all dates until end of month
df_pr_addition_aux = {
    'ds': ['2018-02-23', '2018-02-24', '2018-02-25', '2018-02-26', '2018-02-27', '2018-02-28'],
    'y': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
}
df_pr_addition = pd.DataFrame.from_dict(df_pr_addition_aux)
df_pr = pd.concat([df_pr, df_pr_addition])
df_pr = df_pr.reset_index()
df_pr.drop('index', axis=1,inplace=True)

In [ ]:
## SET PARAMETERS
START_TRAINING_DATE = pd.to_datetime(df_d_sales_full.date.unique()[0])
END_TRAINING_DATE = "2017-01-01"
END_CV_DATE = "2017-08-01"
END_TEST_DATE = "2017-11-01"
END_HOLDOUT_DATE = "2018-03-01"
FORECAST_HORIZON = 16 # for cv and for predicting in test and holdout to get full month forecasts

In [ ]:
## Define datasets and calculate percentages of train, cross validation and test.
train_data = df_pr[df_pr.ds < END_TRAINING_DATE]
cv_data = df_pr[(df_pr.ds >= END_TRAINING_DATE) & (df_pr.ds < END_CV_DATE)]
test_data = df_pr[(df_pr.ds >= END_CV_DATE) & (df_pr.ds < END_TEST_DATE)]
holdout = df_pr[df_pr.ds >= END_TEST_DATE]
assert len(train_data) + len(cv_data) + len(test_data) + len(holdout) == len(df_pr)
print("Number of data points in train, cv and test: {}, {},  {}".format(len(train_data), \
                                                                       len(cv_data), len(test_data)))
tot_ds = len(train_data) + len(cv_data) + len(test_data)
print("Percentages of dataset (excluding holdout) distributed in training, cv, test: {}%, {}%, {}%"\
              .format(round(len(train_data)/tot_ds*100,1), round(len(cv_data)/tot_ds*100,1), \
                      round(len(test_data)/tot_ds*100,1)))

In [ ]:
def datetime_format(string_date):
    """Converts a string to a datetime object."""
    dt_date = datetime.strptime(string_date, '%Y-%m-%d')
    return dt_date

### CROSS VALIDATION

In [ ]:
## Select number of days for training: it will always be the same throughout cross-validation.
first_day_cv = datetime_format(cv_data.ds.astype(str).unique()[0])
train_length = (first_day_cv - START_TRAINING_DATE).days
train_length

In [ ]:
def get_ymd(string_date,i):
    """Obtains year (i=0), month (i=1) or day (i=2)
    from a string date."""
    if i in [0,1,2]:
        value = int(string_date.split('-')[i])
    else:
        print("Parameter 'i' should be 0 for year, 1 for month, 2 for day.")
    return value

def get_cv_dates_by_yymm(start_date, end_date):
    """Given a start and an end date, calculates all dates inbetween
    and appends them to a list as strings."""
    sdy = get_ymd(start_date,0)
    sdm = get_ymd(start_date,1)
    sdd = get_ymd(start_date,2)
    sdate = date(sdy, sdm, sdd)
    
    edy = get_ymd(end_date,0)
    edm = get_ymd(end_date,1)
    edd = get_ymd(end_date,2)
    edate = date(edy, edm, edd)
    
    delta = edate - sdate
    all_days = []
    for i in range(delta.days + 1):
        day = sdate + timedelta(days=i)
        if day.day in range(15,26):
            all_days.append(str(day))
    return all_days

# Calculate training dates for cross-validation.
cv_dates = []
cv_dates = get_cv_dates_by_yymm(END_TRAINING_DATE, END_CV_DATE)

In [ ]:
# Check CV dates are not holidays: if so, remove them
holidays = df_calendar[df_calendar.holiday == 0].date.astype(str).unique()
cv_dates_final = [x for x in cv_dates if x not in holidays]

In [ ]:
# Remove weekends from holidays dataset to add regressor
df_calendar_holidays = df_calendar[(df_calendar.holiday == 0) & \
                                  (df_calendar.weekday != "Saturday") &\
                                  (df_calendar.weekday !="Sunday")].date.astype(str).unique()

In [ ]:
# REGRESSORS

# Holidays
holidays_regr = pd.DataFrame({
  'holiday': 'country_1_holidays',
  'ds': pd.to_datetime(list(df_calendar_holidays)),
  'lower_window': 0,
  'upper_window': 0,
})

# Days of week with higher sales
def get_higher_sales_days(ds):
    date = pd.to_datetime(ds)
    if date.weekday() in [0,1,2]: #Mon, Tue, Wed
        return 1
    else:
        return 0
    
# Month of the year
def get_month_of_year(ds,m):
    date = pd.to_datetime(ds)
    if date.month == m:
        return 1
    else:
        return 0

# Week of the month
#def get_week_of_month(df):

# LO of the month --> not applied
def add_LO(df, df_LO):
    df['ds'] = df['ds'].apply(datetime_format)
    df['yymm'] = df.ds.map(lambda x: x.strftime('%Y-%m'))
    df_LO.columns = ["LO_monthly_sales", "yymm"]
    df_tot = df.merge(df_LO, on="yymm", how="left")
    df_tot.drop('yymm', axis=1, inplace=True)
    df_tot['ds'] = df_tot.ds.astype(str)
    return df_tot
    
df_pr['is_higher_sales_day'] = df_pr.ds.apply(get_higher_sales_days)
df_pr['is_jan'] = df_pr.ds.apply(get_month_of_year, m=1)
df_pr['is_feb'] = df_pr.ds.apply(get_month_of_year, m=2)
df_pr['is_mar'] = df_pr.ds.apply(get_month_of_year, m=3)
df_pr['is_apr'] = df_pr.ds.apply(get_month_of_year, m=4)
df_pr['is_may'] = df_pr.ds.apply(get_month_of_year, m=5)
df_pr['is_jun'] = df_pr.ds.apply(get_month_of_year, m=6)
df_pr['is_jul'] = df_pr.ds.apply(get_month_of_year, m=7)
df_pr['is_aug'] = df_pr.ds.apply(get_month_of_year, m=8)
df_pr['is_sep'] = df_pr.ds.apply(get_month_of_year, m=9)
df_pr['is_oct'] = df_pr.ds.apply(get_month_of_year, m=10)
df_pr['is_nov'] = df_pr.ds.apply(get_month_of_year, m=11)
df_pr['is_dec'] = df_pr.ds.apply(get_month_of_year, m=12)
#df_pr = add_LO(df_pr, df_lo_sales)

df_pr.head(3)

In [ ]:
# SET PARAMETER GRID: 
# We add here the prior scale and the fourier mode in order to tweak width and height of seasonality bumps
parameter_grid = {
    'prior_scale_regressor': [0.5, 1],
    'fourier_order': [5, 10]
}

In [ ]:
# Create functions to add regressors to prophet model and to future predictions
def get_model(fourier_mode, prior_scale):
    """Sets the model according to regressors, seasonalitiers 
    and holidays chosen. Outputs the model."""
    
    m = Prophet(seasonality_mode = 'additive', weekly_seasonality= False,  daily_seasonality = False, 
                   yearly_seasonality = False, holidays=holidays_regr)
    m.add_seasonality(period=7, name='week', fourier_order = fourier_mode, mode = 'additive')
    m.add_seasonality(period=30.5, name='month', fourier_order = fourier_mode, mode = 'additive')
    m.add_seasonality(period=365.25, name='year', fourier_order = fourier_mode, mode = 'additive')
    m.add_regressor('is_higher_sales_day',prior_scale=prior_scale, mode='additive')
    m.add_regressor('is_jan', prior_scale=prior_scale, mode='additive')
    m.add_regressor('is_feb', prior_scale=prior_scale, mode='additive')
    m.add_regressor('is_mar', prior_scale=prior_scale, mode='additive')
    m.add_regressor('is_apr', prior_scale=prior_scale, mode='additive')
    m.add_regressor('is_may', prior_scale=prior_scale, mode='additive')
    m.add_regressor('is_jun', prior_scale=prior_scale, mode='additive')
    m.add_regressor('is_jul', prior_scale=prior_scale, mode='additive')
    m.add_regressor('is_aug', prior_scale=prior_scale, mode='additive')
    m.add_regressor('is_sep', prior_scale=prior_scale, mode='additive')
    m.add_regressor('is_oct', prior_scale=prior_scale, mode='additive')
    m.add_regressor('is_nov', prior_scale=prior_scale, mode='additive')
    m.add_regressor('is_dec', prior_scale=prior_scale, mode='additive')
    #m.add_regressor('LO_monthly_sales', prior_scale=prior_scale, mode='additive')
    return m

def get_future_preds(m, df, forecast_horizon, month_cv_date=None):
    """Given a model (m) and a forecast horizon, generates future dataset
    and predictions. The original df is given in order to filter our any future
    date that Prophet includes (by setting business days for example), but
    that would not be included in the original dataset.
    month_cv_date is used to further filter future predictions to the month 
    of the cutoff."""
    
    future_aux = m.make_future_dataframe(periods = forecast_horizon, freq = "B") #freq = "B" for business days
    future_aux = future_aux[future_aux.ds.isin(df.ds.unique())] # filter out any other day which is not in original series, just in case
    future_aux["month"] = pd.to_datetime(future_aux.ds).apply(lambda d: d.month) # keep only dates within same month of cv_date
    
    if month_cv_date:
        future = future_aux[future_aux.month == month_cv_date][['ds']]
    else:
        future = future_aux
        
    future['is_higher_sales_day'] = df_pr.ds.apply(get_higher_sales_days)
    future['is_jan'] = future.ds.apply(get_month_of_year, m=1)
    future['is_feb'] = future.ds.apply(get_month_of_year, m=2)
    future['is_mar'] = future.ds.apply(get_month_of_year, m=3)
    future['is_apr'] = future.ds.apply(get_month_of_year, m=4)
    future['is_may'] = future.ds.apply(get_month_of_year, m=5)
    future['is_jun'] = future.ds.apply(get_month_of_year, m=6)
    future['is_jul'] = future.ds.apply(get_month_of_year, m=7)
    future['is_aug'] = future.ds.apply(get_month_of_year, m=8)
    future['is_sep'] = future.ds.apply(get_month_of_year, m=9)
    future['is_oct'] = future.ds.apply(get_month_of_year, m=10)
    future['is_nov'] = future.ds.apply(get_month_of_year, m=11)
    future['is_dec'] = future.ds.apply(get_month_of_year, m=12)
    return future
    

In [ ]:
## Ad-hoc cross-validation function. 

from datetime import datetime, timedelta
def cross_val_fct(df, cv_dates, train_length, forecast_horizon, pred_type): 
    """Takes in list of dates (cv_dates), performs for loop where
    for each one with the decided number of training days (train_length) is used
    to forecasts a number of forecast_horizon dates.
    pred_type can be 'cv' or 'test' (used to filter out predictions before the first prediction 
    date, which is not done for test sets).
    The loop aggregates the prediction results into a dataframe (containing cutoff date, prediction dates
    for each cutoff date, and the prediction results. This dataframe is the output.
    Notice the cutoff is not included in the prediction, but rather in the training."""
    
    results_cv = pd.DataFrame()
    for i in cv_dates:
        # Set always only the same number of days for training:
        cutoff = str((datetime_format(i) - timedelta(days=1)).date()) # say 2017-01-14
        first_pred_day = i # 2017-01-15
        first_training_date = str((datetime_format(first_pred_day) - timedelta(days=train_length)).date())
        
        # Set train and test sets within cross validation
        print("For cross-validation starting on {}, training starts on {}".format(first_pred_day, first_training_date))
        cv_train_data = df[(df.ds < first_pred_day) & (df.ds >= first_training_date)]
        cv_test_data = df[df.ds >= first_pred_day]
        print("Predicting for day {}".format(first_pred_day))
        
        # Train models (we fix here the prior scale and the fourier mode, but these can be turned into parameters
        # for the parameter grid).
        prior_scale = 0.5
        fourier_mode = 5
        m = get_model(fourier_mode, prior_scale)
        m.fit(cv_train_data)
        
        # Make future predictions
        month_cv_date = int(i.split('-')[1])
        future = get_future_preds(m, df, forecast_horizon, month_cv_date)
        
        # Predicting
        print("Predicting..")
        forecast = m.predict(future)
        #fig = m.plot_components(forecast) # plot if wanted
        if pred_type == "cv":
            to_add = forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]][(forecast.ds >= first_pred_day)]
        elif pred_type == "test":
            to_add = forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]]
        else:
            print("Parameter 'pred_type' must be either 'cv' or 'test'.")
        to_add['cutoff'] = cutoff
        
        # concatenate results based on horizon
        results_cv = pd.concat([results_cv, to_add])
    return results_cv

In [ ]:
# Forecasts for cross validation:
fcst = cross_val_fct(df_pr, cv_dates_final, train_length, FORECAST_HORIZON, 'cv')

In [ ]:
# Add the information about the horizon days
fcst.cutoff = pd.to_datetime(fcst.cutoff, format='%Y-%m-%d')
df_pr.ds = pd.to_datetime(df_pr.ds, format='%Y-%m-%d')
fcst = fcst.merge(df_pr, on="ds", how="left")
fcst["horizon"] = fcst["ds"] - fcst["cutoff"]
fcst["horizon"] = fcst["horizon"].apply(lambda d: d.days)

In [ ]:
fcst[fcst.cutoff == "2017-01-15"]

In [ ]:
# Use Prophet metrics to calculate rolling mae and mse as a function of the horizon.
from fbprophet.diagnostics import performance_metrics
from fbprophet.diagnostics import cross_validation
performance_metrics(fcst, rolling_window=4./ fcst.shape[0], metrics=['mae', 'mse'])

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
plot_cross_validation_metric(fcst, metric = 'mae')

## PREDICT IN TEST DATA

In [ ]:
def get_prediction_dates(start_date, end_date, holidays):
    """Given start date and an end date, generate all 
    the dates in between. Then filters out holidays."""
    
    prediction_dates = []
    prediction_dates = get_cv_dates_by_yymm(start_date, end_date)
    prediction_dates_final = [x for x in prediction_dates if x not in holidays]
    return prediction_dates_final

def get_n_training_dd(prediction_dates_final):
    """ Outputs number of training dates give the prediction dates."""
    
    from datetime import datetime, timedelta
    first_day_test = datetime_format(min(prediction_dates_final))
    train_length = (first_day_test - START_TRAINING_DATE).days
    return train_length

def get_predictions(df, pred_dates, forecast_horizon): 
    """Outputs all models, future datasets and prediction results datasets
    obtained for forecasting a forecast_horizon number of days for each day in pred_dates."""
    all_models = {}
    all_futures = {}
    results_pred = pd.DataFrame()
    for i in pred_dates:
        # Set always only the same number of days for training:
        cutoff = str((datetime_format(i) - timedelta(days=1)).date()) # say 2017-01-14
        first_pred_day = i # 2017-01-15
        first_training_date = START_TRAINING_DATE # always keep the beginning to get maximum out of training
        
        # Set train and test sets 
        train_data = df[(df.ds < first_pred_day) & (df.ds >= first_training_date)]
        test_data = df[df.ds >= first_pred_day]
        print("Predicting for day {} using all training set available".format(first_pred_day))
        
        # Train models # comentar
        prior_scale = 0.5
        fourier_mode = 5
        m = get_model(fourier_mode, prior_scale)
        m.fit(train_data)
        
        # Make future predictions
        future = get_future_preds(m, df, forecast_horizon)
        
        # Predicting
        print("Predicting..")
        forecast = m.predict(future)
        #fig = m.plot_components(forecast) # plot if wanted
        to_add = forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]]
        to_add['cutoff'] = cutoff
        
        # Concatenate results based on horizon, concatenate also all models and future datasets (one of both for each predicted date)
        results_pred = pd.concat([results_pred, to_add])
        all_models[first_pred_day] = m
        all_futures[first_pred_day] = future
    return all_models, all_futures, results_pred

In [ ]:
def get_min_test_date(test_dates_final):
    """ Returns the first day of the month of
    test_dates_final."""
    
    ref_date = min(test_dates_final)
    min_test_forecast_date = ref_date[0:8] + "01"
    return min_test_forecast_date

In [ ]:
# Get all predictiond dates
test_dates_final = get_prediction_dates(END_CV_DATE, END_TEST_DATE, holidays)
df_pr.ds = pd.to_datetime(df_pr.ds, format='%Y-%m-%d')

In [ ]:
# Predict for test set
test_m, test_future, test_forecast = get_predictions(df_pr, test_dates_final, FORECAST_HORIZON)

In [ ]:
# Merge test forecasts to original dataset to get real daily value.
test_forecast.cutoff = pd.to_datetime(test_forecast.cutoff, format='%Y-%m-%d')
#df_pr.ds = pd.to_datetime(df_pr.ds, format='%Y-%m-%d')
test_forecast = test_forecast.merge(df_pr, on="ds", how="left")

In [ ]:
# Filter dataset to get only results starting at the beginning of the month of the test set. 
# This is needed to calculate full month forecast.
min_test_forecast_date = get_min_test_date(test_dates_final)
test_forecast_cut = test_forecast[(test_forecast.ds >= min_test_forecast_date)]
test_forecast_cut.head()

In [ ]:
# Add necessary columns used to filter results (specifically, results only for the month of the cutoff date, 
# otherwise we would be adding up values from other months).
test_forecast_cut['month_cutoff'] = test_forecast_cut['cutoff'].apply(lambda x: x.month)
test_forecast_cut['year_cutoff'] = test_forecast_cut['cutoff'].apply(lambda x: x.year)
test_forecast_cut['month_ds'] = test_forecast_cut['ds'].apply(lambda x: x.month)
test_forecast_cut['year_ds'] = test_forecast_cut['ds'].apply(lambda x: x.year)
test_forecast_cut = test_forecast_cut[(test_forecast_cut.month_cutoff == test_forecast_cut.month_ds) & \
                                           (test_forecast_cut.year_cutoff == test_forecast_cut.year_ds)]

### Method 1: Get predictions using samples to estimate errors for aggregate forecasts

Say we want the full-month prediction on the prediction date 2017-11-16. For this date the next 16 days will be forecasted. For each of these days, we can perform predictive samples, which are 1000 sample predictions for that prediction date (instead of only one forecast).

The procedure is: 
- Take all the predicted values for the prediction dates within the month (in this case November), and aggregate them (sum) —> now we have 1000 aggregated values (a distribution of the aggregated values). 
- Of these 1000 aggregations, the mean is taken to consider the prediction value, and the 10th percentile and 90th percentiles are taken to consider the upper and lower values (80% uncertainty boundary). 

For each prediction date, aggregate over the remaining portion of the month.

In [ ]:
def get_aggregate_forecast_error(m, future, pred_dates):
    """ Obtains predictive samples. Takes in the model, the future dataset 
    and a list prediction dates to loop over."""
    
    all_predictions_errors = pd.DataFrame()
    for i in pred_dates:
        #  Create samples of predictions for every specific day in future
        samples = m[i].predictive_samples(future[i])

        # Creates a DF where each column is 1 prediction for that specific day
        samples_df = pd.DataFrame.from_records(samples["yhat"])
        samples_df['date'] = future[i]['ds']
        samples_df['month'] = samples_df['date'].apply(lambda x: x.month)
        month_min_fcst_date = int(i.split('-')[1])
        # 
        samples_df = samples_df[(samples_df.date >= i) & (samples_df.month == month_min_fcst_date)]   
        
        # The mean of each column is thus our yhat
        rest_of_month_predict = samples_df.groupby("month").sum().mean(axis=1)
        rest_of_month_predict = rest_of_month_predict.reset_index()
        rest_of_month_predict['prediction_day'] = i
        rest_of_month_predict.rename(columns={0: "yhat"}, inplace=True)
        rest_of_month_predict = rest_of_month_predict[['prediction_day', 'month', 'yhat']]
        #rest_of_month_predict['start_of_week'] = samples_df.groupby("week_of_year").date.min().reset_index().date.tolist()

        # Upper and lower values of yhat are computed following fbprophet's approach
        upper_lower = samples_df.groupby("month").sum().reset_index()
        rest_of_month_predict['yhat_lower'] = upper_lower.apply(lambda x: np.percentile(x, 10), axis=1).tolist()
        rest_of_month_predict['yhat_upper'] = upper_lower.apply(lambda x: np.percentile(x, 90), axis=1).tolist()
        
        all_predictions_errors = pd.concat([all_predictions_errors, rest_of_month_predict])
    return all_predictions_errors

In [ ]:
all_rolled_preds = get_aggregate_forecast_error(test_m, test_future, test_dates_final)

In [ ]:
all_rolled_preds.head()

In [ ]:
# Calculates real values up to the cutoff (included):
all_real = test_forecast_cut[test_forecast_cut.ds <= test_forecast_cut.cutoff][['ds', 'y', 'cutoff']]
all_real = test_forecast_cut[(test_forecast_cut.ds <= test_forecast_cut.cutoff) & \
                               ((test_forecast_cut.month_cutoff == test_forecast_cut.month_ds) & \
                                           (test_forecast_cut.year_cutoff == test_forecast_cut.year_ds))][['ds', 'y', 'cutoff']]
all_real['prediction_day'] = all_real['cutoff'].apply(lambda x: (x + timedelta(days=1)).date())
all_rolled_real = all_real.groupby('prediction_day').sum().reset_index()
all_rolled_real['prediction_day'] = all_rolled_real.prediction_day.astype(str)
all_rolled_real.head()

In [ ]:
# Merge rolled predictions with rolled real values to obtain the monthly forecast for each day.
test_aggr_forecasts = all_rolled_preds.merge(all_rolled_real, on="prediction_day", how='outer')
test_aggr_forecasts['monthly_forecast'] = test_aggr_forecasts['y'] + test_aggr_forecasts['yhat']
test_aggr_forecasts = test_aggr_forecasts[['prediction_day', 'monthly_forecast', 'yhat_lower', 'yhat_upper']]
test_aggr_forecasts.columns = ['prediction_day', 'monthly_forecast', 'monthly_forecast_lower', 'monthly_forecast_upper']

In [ ]:
test_aggr_forecasts.head(3)

In [ ]:
def get_fcst_agg_actual_results(test_aggr_forecasts,df_actual_sales):
    """Calculates custom variance (percentage difference between 
    forecasted and real value)."""
    
    test_aggr_forecasts.prediction_day = pd.to_datetime(test_aggr_forecasts.prediction_day, format='%Y-%m-%d')
    test_aggr_forecasts['month'] = test_aggr_forecasts['prediction_day'].apply(lambda x: x.month)
    test_aggr_forecasts['year'] = test_aggr_forecasts['prediction_day'].apply(lambda x: x.year)
    test_aggr_forecasts['yymm'] = 100*test_aggr_forecasts.year + test_aggr_forecasts.month
    test_aggr_forecasts['yymm'] = pd.to_datetime(test_aggr_forecasts.yymm, format='%Y%m').map(lambda x: x.strftime('%Y-%m'))
    
    df_actual_sales.yymm = pd.to_datetime(df_actual_sales.yymm, format='%Y-%m').map(lambda x: x.strftime('%Y-%m'))
    test_forecast_comparison = test_aggr_forecasts.merge(df_actual_sales, on="yymm", how='left')
    test_forecast_comparison["delta"] = test_forecast_comparison.monthly_forecast - test_forecast_comparison.monthly_sales
    test_forecast_comparison["variance"] = test_forecast_comparison.delta/test_forecast_comparison.monthly_sales
    return test_forecast_comparison

In [ ]:
test_forecast_comparison = get_fcst_agg_actual_results(test_aggr_forecasts,df_actual_sales)
test_forecast_comparison.head(12)

In [ ]:
test_forecast_comparison

In [ ]:
# Plot custom variance results for test set (where we actually have actual results)
test_forecast_comparison['dd'] = test_forecast_comparison.prediction_day.apply(lambda x: x.day)

In [ ]:
plt.figure(figsize=(18,9))
plt.plot(test_forecast_comparison[test_forecast_comparison.yymm == '2017-08']["dd"], test_forecast_comparison[test_forecast_comparison.yymm == '2017-08']["variance"], linestyle="-", color='#346BAE')
plt.plot(test_forecast_comparison[test_forecast_comparison.yymm == '2017-09']["dd"], test_forecast_comparison[test_forecast_comparison.yymm == '2017-09']["variance"], linestyle="-", color='#D55632')
plt.plot(test_forecast_comparison[test_forecast_comparison.yymm == '2017-10']["dd"], test_forecast_comparison[test_forecast_comparison.yymm == '2017-10']["variance"], linestyle="-", color='#E09E3F')
plt.axhline(y=0.0, color='#d0d0d0', linestyle='--')
plt.xlabel=('Dates')
plt.ylabel=('Custom Variance')
plt.legend(["2017-08", "2017-09", "2017-10"])
plt.show();

### Method 2: Get predictions as simple sum of prophet forecasts (no error estimation)

In [ ]:
def get_agg_pred_by_cutoff(test_forecast_cut):
    #fcst.cutoff = pd.to_datetime(fcst.cutoff, format='%Y-%m-%d')
    test_forecast_cut['y_to_accum'] = np.where(test_forecast_cut.ds <= test_forecast_cut.cutoff, \
                                               test_forecast_cut.y, test_forecast_cut.yhat)
    test_forecast_cut['y_to_accum_upper'] = np.where(test_forecast_cut.ds <= test_forecast_cut.cutoff, \
                                               test_forecast_cut.y, test_forecast_cut.yhat)
    test_forecast_cut['month'] = test_forecast_cut['ds'].apply(lambda x: x.month)
    test_forecast_cut['year'] = test_forecast_cut['ds'].apply(lambda x: x.year)
    test_forecast_cut['yymm'] = 100*test_forecast_cut.year + test_forecast_cut.month
    test_forecast_cut['yymm'] = pd.to_datetime(test_forecast_cut.yymm, format='%Y%m').map(lambda x: x.strftime('%Y-%m'))

    test_forecast_cut_grpd = test_forecast_cut.groupby(["cutoff","yymm"]).agg({'y_to_accum': ['sum']})
    test_forecast_cut_grpd.columns = ['_'.join(col) for col in test_forecast_cut_grpd.columns]
    test_forecast_cut_grpd = test_forecast_cut_grpd.reset_index()
    return test_forecast_cut_grpd
        
def get_fcst_agg_actual_results(test_forecast_cut_grpd,df_actual_sales):
    test_forecast_comparison = test_forecast_cut_grpd.merge(df_actual_sales, on="yymm", how='left')
    test_forecast_comparison["delta"] = test_forecast_comparison.y_to_accum_sum - test_forecast_comparison.monthly_sales
    test_forecast_comparison["variance"] = test_forecast_comparison.delta/test_forecast_comparison.monthly_sales
    return test_forecast_comparison

In [ ]:
test_forecast_cut_grpd = get_agg_pred_by_cutoff(test_forecast_cut)
test_forecast_comparison_2 = get_fcst_agg_actual_results(test_forecast_cut_grpd,df_actual_sales)

In [ ]:
test_forecast_comparison_2['dd'] = test_forecast_comparison_2.cutoff.apply(lambda x: x.day)
test_forecast_comparison_2

In [ ]:
# Plot custom variance: we see it doesn't change compared to the previous.
plt.figure(figsize=(18,9))
plt.plot(test_forecast_comparison_2[test_forecast_comparison_2.yymm == '2017-08']["dd"], test_forecast_comparison_2[test_forecast_comparison_2.yymm == '2017-08']["variance"], linestyle="-", color='#346BAE')
plt.plot(test_forecast_comparison_2[test_forecast_comparison_2.yymm == '2017-09']["dd"], test_forecast_comparison_2[test_forecast_comparison_2.yymm == '2017-09']["variance"], linestyle="-", color='#D55632')
plt.plot(test_forecast_comparison_2[test_forecast_comparison_2.yymm == '2017-10']["dd"], test_forecast_comparison_2[test_forecast_comparison_2.yymm == '2017-10']["variance"], linestyle="-", color='#E09E3F')
plt.axhline(y=0.0, color='#d0d0d0', linestyle='--')
plt.xlabel=('Dates')
plt.ylabel=('Custom Variance')
plt.legend(["2017-08", "2017-09", "2017-10"])
plt.show();

### Comparison to baseline model

In the test set we also compare to a baseline (t0 or naïve) model. The baseline model simply takes the value of the previous day. Here we take all the training and cross validation sets to train, and we predict on the horizon of all test set.
We then calculate MAE and MSE of the results.

In [ ]:
# Define baseline horizon (all test set)
baseline_date = min(test_data.ds)
from datetime import datetime, timedelta
first_day_baseline = datetime_format(baseline_date)
train_length = (first_day_baseline - START_TRAINING_DATE).days
train_length #1048

baseline_horizon = (datetime_format(END_TEST_DATE) - first_day_baseline).days
print(first_day_baseline)

In [ ]:
baseline_horizon

In [ ]:
#test_data
#baseline_forecast = cross_val_fct(df_pr, baseline_date, train_length, FORECAST_HORIZON, 'test')
baseline_train_data = df_pr[(df_pr.ds < first_day_baseline)]
baseline_test_data = df_pr[df_pr.ds >= first_day_baseline]
m = get_model(5, 0.5)
m.fit(baseline_train_data)
future = get_future_preds(m, df_pr, baseline_horizon)
baseline_pred = m.predict(future)
m.plot(baseline_pred)
m.plot_components(baseline_pred)


In [ ]:
# Define errors
baseline_pred_short = baseline_pred.copy()
baseline_pred_short = baseline_pred_short.merge(df_pr, on="ds", how="left")[["ds", "yhat","y"]]
baseline_pred_short["ybase"] = baseline_pred_short.y.shift(1) # We imagine to make as a prediction the value of the day before
baseline_pred_short["ae_prophet"] = abs(baseline_pred_short.yhat - baseline_pred_short.y)
baseline_pred_short["ae_baseline"] = abs(baseline_pred_short.ybase - baseline_pred_short.y)
baseline_pred_short["se_prophet"] = (baseline_pred_short.yhat - baseline_pred_short.y)**2
baseline_pred_short["se_baseline"] = (baseline_pred_short.ybase - baseline_pred_short.y)**2
baseline_pred_short = baseline_pred_short[baseline_pred_short.ds >= "2015-01-02"]

# ERRORS
mae_train_prophet = round(baseline_pred_short[baseline_pred_short.ds < first_day_baseline].ae_prophet.mean(),2)
mae_train_baseline = round(baseline_pred_short[baseline_pred_short.ds < first_day_baseline].ae_baseline.mean(),2)
mae_test_prophet = round(baseline_pred_short[baseline_pred_short.ds >= first_day_baseline].ae_prophet.mean(),2)
mae_test_baseline = round(baseline_pred_short[baseline_pred_short.ds >= first_day_baseline].ae_baseline.mean(),2)

mse_train_prophet = round(baseline_pred_short[baseline_pred_short.ds < first_day_baseline].se_prophet.mean(),2)
mse_train_baseline = round(baseline_pred_short[baseline_pred_short.ds < first_day_baseline].se_baseline.mean(),2)
mse_test_prophet = round(baseline_pred_short[baseline_pred_short.ds >= first_day_baseline].se_prophet.mean(),2)
mse_test_baseline = round(baseline_pred_short[baseline_pred_short.ds >= first_day_baseline].se_baseline.mean(),2)

rmse_train_prophet = round(math.sqrt(mse_train_prophet),2)
rmse_train_baseline = round(math.sqrt(mse_train_baseline),2)
rmse_test_prophet = round(math.sqrt(mse_test_prophet),2)
rmse_test_baseline = round(math.sqrt(mse_test_baseline),2)

print(mae_train_prophet, mae_train_baseline)
print(mae_test_prophet, mae_test_baseline)
print(mse_train_prophet, mse_train_baseline)
print(mse_test_prophet, mse_test_baseline)  
print(rmse_train_prophet, rmse_train_baseline)
print(rmse_test_prophet, rmse_test_baseline)  # OK, seems baseline errors are much bigger!

In [ ]:
# Create dataframe for errors in order to plot them
errors_baseline_dix_1 = {
    'dataset': ['train', 'test'], 
    'MAE_Prophet': [mae_train_prophet, mae_test_prophet],
    'MAE_Baseline': [mae_train_baseline, mae_test_baseline],
    'MSE_Prophet': [mse_train_prophet, mse_test_prophet],
    'MSE_Baseline': [mse_train_baseline, mse_test_baseline]
}
errors_baseline_dix = {
    'metric': ['MAE_Prophet', "MAE_Baseline", "MSE_Prophet", "MSE_Baseline"],
    'train': [mae_train_prophet, mae_train_baseline, mse_train_prophet, mse_train_baseline], 
    'test': [mae_test_prophet, mae_test_baseline, mse_test_prophet, mse_test_baseline]
}
errors_baseline_df = pd.DataFrame.from_dict(errors_baseline_dix)
#errors_baseline_df.train = errors_baseline_df.train.apply(lambda x: np.log(x))
#errors_baseline_df.test = errors_baseline_df.test.apply(lambda x: np.log(x))
errors_baseline_df = errors_baseline_df.set_index("metric")

In [ ]:
# bar plot
print(errors_baseline_df)
ax = errors_baseline_df[errors_baseline_df.index.isin(["MAE_Prophet", "MAE_Baseline"])].plot.bar(rot=0, color=["#346BAE", "#E09E3F"],figsize=(10,6))
ax = errors_baseline_df[errors_baseline_df.index.isin(["MSE_Prophet", "MSE_Baseline"])].plot.bar(rot=0, color=["#346BAE", "#E09E3F"],figsize=(10,6))

## PREDICT IN HOLDOUT DATA

We now predict for the holdout, in the same way we did for the test set (we will use method 1 in test set where we can calculate the uncertainty boundaries for the aggregates as well).

In [ ]:
df_calendar_addition_aux = {
    'date': ['2018-02-24', '2018-02-25', '2018-02-26', '2018-02-27', '2018-02-28'],
    'yymm': ['2018-02', '2018-02', '2018-02', '2018-02', '2018-02'],
    'weekday': ['Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday'],
    'year': [2018, 2018, 2018, 2018, 2018],
    'month': [2,2,2,2,2],
    'month_lit': ['Feb', 'Feb', 'Feb', 'Feb', 'Feb'],
    'day': [24, 25, 26, 27, 28],
    'holiday': [0,0,1,1,1],
    'is_real_day_sale': [0,0,0,0,1]
} # we had erased these dates but we need them, so we add them here.
df_calendar_addition = pd.DataFrame.from_dict(df_calendar_addition_aux)
df_calendar_complete = pd.concat([df_calendar, df_calendar_addition])
holidays = df_calendar_complete[df_calendar_complete.holiday == 0].date.astype(str).unique()

In [ ]:
# Get all holdout prediction dates
holdout_dates_final = get_prediction_dates(END_TEST_DATE, END_HOLDOUT_DATE, holidays)

In [ ]:
# Predict
ho_m, ho_future, holdout_forecast = get_predictions(df_pr, holdout_dates_final, FORECAST_HORIZON)

In [ ]:
# Merge to get real values as well
holdout_forecast.cutoff = pd.to_datetime(holdout_forecast.cutoff, format='%Y-%m-%d')
df_pr.ds = pd.to_datetime(df_pr.ds, format='%Y-%m-%d')
holdout_forecast = holdout_forecast.merge(df_pr, on="ds", how="left")

In [ ]:
holdout_forecast

In [ ]:
# Filter out unnecessary dates
min_holdout_forecast_date = get_min_test_date(holdout_dates_final)
holdout_forecast_cut = holdout_forecast[(holdout_forecast.ds >= min_holdout_forecast_date)]
holdout_forecast_cut.head()

In [ ]:
# Create new columns in order to further filter out (see explanation in test set).
holdout_forecast_cut['month_cutoff'] = holdout_forecast_cut['cutoff'].apply(lambda x: x.month)
holdout_forecast_cut['year_cutoff'] = holdout_forecast_cut['cutoff'].apply(lambda x: x.year)
holdout_forecast_cut['month_ds'] = holdout_forecast_cut['ds'].apply(lambda x: x.month)
holdout_forecast_cut['year_ds'] = holdout_forecast_cut['ds'].apply(lambda x: x.year)
holdout_forecast_cut = holdout_forecast_cut[(holdout_forecast_cut.month_cutoff == holdout_forecast_cut.month_ds) & \
                                           (holdout_forecast_cut.year_cutoff == holdout_forecast_cut.year_ds)]

### Get predictions using samples to estimate errors for aggregate forecasts

In [ ]:
# Calculate aggregated predictions
all_rolled_preds = get_aggregate_forecast_error(ho_m, ho_future, holdout_dates_final)

In [ ]:
all_rolled_preds

In [ ]:
# Calculate aggregates for real values as well
all_real = holdout_forecast_cut[(holdout_forecast_cut.ds <= holdout_forecast_cut.cutoff) & \
                               ((holdout_forecast_cut.month_cutoff == holdout_forecast_cut.month_ds) & \
                                           (holdout_forecast_cut.year_cutoff == holdout_forecast_cut.year_ds))][['ds', 'y', 'cutoff']]
all_real['prediction_day'] = all_real['cutoff'].apply(lambda x: (x + timedelta(days=1)).date())
all_rolled_real = all_real.groupby('prediction_day').sum().reset_index()
all_rolled_real['prediction_day'] = all_rolled_real.prediction_day.astype(str)
#all_rolled_real
all_rolled_real.head(5)

In [ ]:
# Merge aggregated real and forecasted values
holdout_aggr_forecasts = all_rolled_preds.merge(all_rolled_real, on="prediction_day", how='outer')
holdout_aggr_forecasts['monthly_forecast'] = holdout_aggr_forecasts['y'] + holdout_aggr_forecasts['yhat']
holdout_aggr_forecasts = holdout_aggr_forecasts[['prediction_day', 'monthly_forecast', 'yhat_lower', 'yhat_upper']]
holdout_aggr_forecasts.columns = ['prediction_day', 'monthly_forecast', 'monthly_forecast_lower', 'monthly_forecast_upper']
holdout_aggr_forecasts.head()

In [ ]:
holdout_aggr_forecasts